<a href="https://colab.research.google.com/github/csaybar/EEwPython/blob/master/1_4_Features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<!--COURSE_INFORMATION-->
<img align="left" style="padding-right:10px;" src="https://sitejerk.com/images/google-earth-logo-png-5.png" width=5% >
<img align="right" style="padding-left:10px;" src="https://colab.research.google.com/img/colab_favicon_256px.png" width=6% >


>> *This notebook is part from the free course [EEwPython](https://github.com/csaybar/EEwPython); the content is available [on GitHub](https://github.com/csaybar/EEwPython)* and released under the [Apache 2.0 License](https://www.gnu.org/licenses/gpl-3.0.en.html).



<!--NAVIGATION-->
 < [Image & ImageCollection](1.3_Images.ipynb) | [Contents](index.ipynb) |  [Conditional statements](1.5_Cond_Statements.ipynb)>

<a href="https://colab.research.google.com/github/csaybar/EEwPython/blob/master/1.4_Features.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>

In [0]:
#@title # Geometry, Features & FeaturesCollection
from IPython.display import HTML
HTML('<center><iframe width="560" height="315" src="https://www.youtube.com/embed/LzxQH0Ze0iI" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe></center>')

In [0]:
#@title Function to visualize thumbnail URL
from IPython.display import Image
from IPython.display import HTML

def display_image(url,title="",CSS_row=False):
  img_link = "<center> <h1>%s</h1> <img title='%s' src= '%s'></center>" % (title,title,url)
  return HTML(img_link)

### Connecting GEE, Colab and Google Drive

- **Colab & Google Drive synchronization**

In [0]:
#from google.colab import drive
#drive.mount('/content/drive')

- **Colab & Earth Engine synchronization**

In [0]:
!pip install earthengine-api #Firstly we need the earth-engine API

In [0]:
!earthengine authenticate 

In [0]:
import ee
ee.Initialize()

### 1.  Geometry overview

Earth Engine handles vector data with the `Geometry` type. The [GeoJSON spec](https://geojson.org/geojson-spec.html) describes in detail the type of geometries supported by Earth Engine, including `Point` (a list of coordinates in some projection), `LineString` (a list of points), `LinearRing` (a closed `LineString`), and `Polygon` (a list of `LinearRings` where the first is a shell and subsequent rings are holes). Earth Engine also supports `MultiPoint`, `MultiLineString`, and `MultiPolygon`. The GeoJSON GeometryCollection is also supported, although it has the name `MultiGeometry` within Earth Engine.


#### Creating Geometry objects
 To create a Geometry programmatically, provide the constructor with the proper list(s) of coordinates. For example:

In [0]:
point = ee.Geometry.Point([1.5, 1.5])
lineString = ee.Geometry.LineString([[-35, -10], [35, -10], [35, 10], [-35, 10]])
linearRing = ee.Geometry.LinearRing([[-35, -10], [35, -10], [35, 10], [-35, 10], [-35, -10]])
rectangle = ee.Geometry.Rectangle([-40, -20, 40, 20])
polygon = ee.Geometry.Polygon([[[-5, 40], [65, 40], [65, 60], [-5, 60], [-5, 60]]])

In the previous examples, note that the distinction between a `LineString` and a `LinearRing` is that the `LinearRing` is “closed” by having the same coordinate at both the start and end of the list.

An individual `Geometry` may consist of multiple geometries. To break a multi-part `Geometry` into its constituent geometries, use `geometry.geometries()`. For example:

In [0]:
# Create a multi-part feature.
multiPoint = ee.Geometry.MultiPoint([[-121.68, 39.91], [-97.38, 40.34]])

# Get the individual geometries as a list.
geometries = multiPoint.geometries()

# Get each individual geometry from the list and print it.
pt1 = geometries.get(0)
pt2 = geometries.get(1)
print('Point 1', pt1)
print('Point 2', pt2)     

#### Geometry information and metadata

To view information about a geometry, print it. To access the information programmatically, Earth Engine provides several methods. For example, to get information about the polygon created previously, use:




In [0]:
print('Polygon printout: ', polygon.getInfo())

# Print polygon area in square kilometers.
print('Polygon area: ', polygon.area().divide(1000 * 1000).getInfo())

# Print polygon perimeter length in kilometers.
print('Polygon perimeter: ', polygon.perimeter().divide(1000).getInfo())

# Print the geometry as a GeoJSON string.
print('Polygon GeoJSON: ', polygon.toGeoJSONString())

# Print the GeoJSON 'type'
print('Geometry type: ', polygon.type().getInfo())

# Print the coordinates as lists
print('Polygon coordinates: ', polygon.coordinates().getInfo())

# Print whether the geometry is geodesic.
print('Geodesic? ', polygon.geodesic().getInfo())      

Polygon printout:  {'type': 'Polygon', 'coordinates': [[[-5.0, 40.0], [65.0, 40.0], [65.0, 60.0], [-5.0, 60.0], [-5.0, 60.0], [-5.0, 40.0]]]}
Polygon area:  9918985.748240016
Polygon perimeter:  13979.88727044261
Polygon GeoJSON:  {"type": "Polygon", "coordinates": [[[-5, 40], [65, 40], [65, 60], [-5, 60], [-5, 60]]], "evenOdd": true}
Geometry type:  Polygon
Polygon coordinates:  [[[-5.0, 40.0], [65.0, 40.0], [65.0, 60.0], [-5.0, 60.0], [-5.0, 60.0], [-5.0, 40.0]]]
Geodesic?  True


Observe that the perimeter (or length) of a geometry is returned in meters and the area is returned in square meters unless a projection is specified. By default, the computation is performed on the WGS84 spheroid and the result is computed in meters or square meters.

### 2. Feature Overview

A `Feature` in Earth Engine is defined as a GeoJSON `Feature`. Specifically, a `Feature` is an object with a geometry property storing a `Geometry` object (or null) and a `properties` property storing a dictionary of other properties.

#### Creating Feature objects
To create a `Feature`, provide the constructor with a `Geometry` and (optionally) a dictionary of other properties. For example:

In [0]:
# Create an ee.Geometry.
polygon = ee.Geometry.Polygon([[[-35, -10], [35, -10], [35, 10], [-35, 10], [-35, -10]]])

# Create a Feature from the Geometry.
polyFeature = ee.Feature(polygon, {'foo': 42, 'bar': 'tart'})

As with a `Geometry`, a `Feature` may be printed or added to the map for inspection and visualization:



In [0]:
polyFeature.getInfo()

{'geometry': {'coordinates': [[[-35.0, -10.0],
    [35.0, -10.0],
    [35.0, 10.0],
    [-35.0, 10.0],
    [-35.0, -10.0]]],
  'type': 'Polygon'},
 'properties': {'bar': 'tart', 'foo': 42},
 'type': 'Feature'}

A `Feature` need not have a Geometry and may simply wrap a dictionary of properties. For example:



In [0]:
# Create a dictionary of properties, some of which may be computed values.
dict = {'foo': ee.Number(8).add(88), 'bar': 'nihao'}

# Create a null geometry feature with the dictionary of properties.
nowhereFeature = ee.Feature(None, dict)

In this example, note that the dictionary supplied to the `Feature` contains a computed value. Creating features in this manner is useful for exporting long-running computations with a `Dictionary` result (e.g. `image.reduceRegion()`). See the [FeatureCollections](https://developers.google.com/earth-engine/feature_collections) and [Importing](https://developers.google.com/earth-engine/importing) or [Exporting](https://developers.google.com/earth-engine/exporting) sections for details.

Each `Feature` has one primary `Geometry` stored in the `geometry` property. Additional geometries may be stored in other properties. `Geometry` methods such as intersection and buffer also exist on `Feature` as a convenience for getting the primary `Geometry`, applying the operation, and setting the result as the new primary `Geometry`. The result will retain all the other properties of the `Feature` on which the method is called. There are also methods for getting and setting the non-geometry properties of the `Feature`. For example:

In [0]:
# Make a feature and set some properties.
feature = ee.Feature(ee.Geometry.Point([-122.22599, 37.17605]))\
            .set('genus', 'Sequoia').set('species', 'sempervirens')

# Get a property from the feature.
species = feature.get('species')
print(species.getInfo())

# Set a new property.
feature = feature.set('presence', 1)

# Overwrite the old properties with a new dictionary.
newDict = {'genus': 'Brachyramphus', 'species': 'marmoratus'}
feature = feature.set(newDict)

# Check the result.
print(feature.getInfo())

sempervirens
{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-122.22599, 37.17605]}, 'properties': {'genus': 'Brachyramphus', 'presence': 1, 'species': 'marmoratus'}}


In the previous example, note that properties can be set with either a key: value pair, or with a dictionary as a JavaScript literal. Also note that `feature.set()` overwrites existing properties.

### 3. FeatureCollection Overview

Groups of related features can be combined into a `FeatureCollection`, to enable additional operations on the entire set such as filtering, sorting and rendering. Besides just simple features (geometry + properties), feature collections can also contain other collections.

#### The FeatureCollection constructor

One way to create a `FeatureCollection` is to provide the constructor with a list of features. The features do not need to have the same geometry type or the same properties. For example:

In [0]:
# Make a list of Features.
features = [
  ee.Feature(ee.Geometry.Rectangle(30.01, 59.80, 30.59, 60.15), {'name': 'Voronoi'}),
  ee.Feature(ee.Geometry.Point(-73.96, 40.781), {'name': 'Thiessen'}),
  ee.Feature(ee.Geometry.Point(6.4806, 50.8012), {'name': 'Dirichlet'})
]

# Create a FeatureCollection from the list and print it.
fromList = ee.FeatureCollection(features)
fromList.getInfo()

{'columns': {'name': 'String', 'system:index': 'String'},
 'features': [{'geometry': {'coordinates': [[[30.01, 59.8],
      [30.59, 59.8],
      [30.59, 60.15],
      [30.01, 60.15],
      [30.01, 59.8]]],
    'type': 'Polygon'},
   'id': '0',
   'properties': {'name': 'Voronoi'},
   'type': 'Feature'},
  {'geometry': {'coordinates': [-73.96, 40.781], 'type': 'Point'},
   'id': '1',
   'properties': {'name': 'Thiessen'},
   'type': 'Feature'},
  {'geometry': {'coordinates': [6.4806, 50.8012], 'type': 'Point'},
   'id': '2',
   'properties': {'name': 'Dirichlet'},
   'type': 'Feature'}],
 'type': 'FeatureCollection'}

Individual geometries can also be turned into a `FeatureCollection` of just one Feature:



In [0]:
# Create a FeatureCollection from a single geometry and print it.
fromGeom = ee.FeatureCollection(ee.Geometry.Point(16.37, 48.225))
fromGeom.getInfo()

{'columns': {'system:index': 'String'},
 'features': [{'geometry': {'coordinates': [16.37, 48.225], 'type': 'Point'},
   'id': '0',
   'properties': {},
   'type': 'Feature'}],
 'type': 'FeatureCollection'}

<!--NAVIGATION-->
 < [Image & ImageCollection](1.3_Images.ipynb) | [Contents](index.ipynb) |  [Conditional statements](1.5_Cond_Statements.ipynb)>

<a href="https://colab.research.google.com/github/csaybar/EEwPython/blob/master/1.4_Features.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>